## Imports

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf


from utils.data_utils import (
    Mode,
)

# Setting seed so that it can be deterministic in initialization
tf.keras.utils.set_random_seed(11)
tf.config.experimental.enable_op_determinism()

## Constants

In [ ]:
N_TRAIN_AGENT = 30000
N_VAL_AGENT = 3000
NUM_TRIAL = 500

### Change me
classification_type = "HRL"  # @param ['2PRL-t', 'HRL', 'PRL'] {allow-input: true}
###

if classification_type == "HRL":
    cognitive_model_to_data_file = {
        Mode.HRL2: ("t", "v", 'test'),
        Mode.HRL2_Bayes: ("t", "v", 'test'),
        Mode.HRL2_StickyBayes: ("t", "v", 'test'),
    }
elif classification_type == "PRL":
    cognitive_model_to_data_file = {
        Mode.PRL2: ("t", "v", 'test'),
        Mode.PRL4: ("t", "v", 'test'),
        Mode.Bayes: ("t", "v", 'test'),
        Mode.StickyBayes: ("t", "v", 'test'),
    }
else:
    cognitive_model_to_data_file = {
        Mode.PRL2: ("t", "v", 'test'),
        Mode.PRL2_intractable: ("t", "v", 'test'),
    }

OUPUT_DIM = len(cognitive_model_to_data_file)

## Read data

In [ ]:
cognitive_model_to_train_data = {}
cognitive_model_to_val_data = {}
for mode, (train_filepath, val_filepath, _) in cognitive_model_to_data_file.items():
    cognitive_model_to_train_data[mode] = pd.read_csv(train_filepath)
    cognitive_model_to_val_data[mode] = pd.read_csv(val_filepath)

## Process Data

In [ ]:
from process_data import process_data

target_trial = 500
train_features, normalized_train_labels = process_data(
    cognitive_model_to_raw_data, N_TRAIN_AGENT, NUM_TRIAL, target_trial
)
val_features, normalized_val_labels = process_data(
    cognitive_model_to_val_data, N_VAL_AGENT, NUM_TRIAL, target_trial
)

In [ ]:
from sklearn.utils import shuffle

train_features, normalized_train_labels = shuffle(
    train_features.numpy(), normalized_train_labels, random_state=0
)
val_features, normalized_val_labels = shuffle(
    val_features.numpy(), normalized_val_labels, random_state=0
)
print(train_features.shape, normalized_train_labels.shape)

## Model Training

### Bayesian Search

In [ ]:
from utils.hypetune import bayesian_search
from model import get_gru_model

# (TODO) move to a config file
param_grid = {
    "input_x": train_features.shape[1],
    "input_y": train_features.shape[2],
    "output_dim": output_dim,
    "units": 64 + hp.randint("units", 128),
    "learning_rate": 3e-4,
    "dropout": hp.uniform("dropout", 0.15, 0.25),
    "dropout1": hp.uniform("dropout1", 0.01, 0.1),
    "dropout2": hp.uniform("dropout2", 0.01, 0.05),
    "epochs": 25,
    "batch_size": 256,
}

best_model, best_params = bayesian_search(
    get_gru_model,
    param_grid,
    train_features,
    normalized_train_labels,
    val_features,
    normalized_val_labels,
)

print(f"Found best parameters {best_params}")

### Training model

In [ ]:
from keras.callbacks import EarlyStopping

callbacks = [EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)]

history = best_model.fit(
    train_features,
    normalized_train_labels,
    epochs=200,
    batch_size=best_params['batch_size'],
    callbacks=callbacks,
    validation_data=(val_features, normalized_val_labels),
    verbose=2,
)

## Model Evaluation

In [ ]:
cognitive_model_to_test_data = {}
for mode, (_, _, test_filepath) in cognitive_model_to_data_file.items():
    cognitive_model_to_test_data[mode] = pd.read_csv(test_filepath)
    
all_test_features, all_test_labels = process_data(
    cognitive_model_to_test_data, N_VAL_AGENT, NUM_TRIAL, all_trials
)
print(all_test_features.shape, all_test_labels.shape)

In [ ]:
accr = best_model.evaluate(all_test_features, all_test_labels)
print("Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}".format(accr[0], accr[1]))

### Classification Report

In [ ]:
from sklearn.metrics import classification_report

prediction = best_model.predict(all_test_features)
y_true, y_pred = np.argmax(all_test_labels, axis=1), np.argmax(
    prediction, axis=1
)

classification_report(
    y_true, y_pred, target_names=cognitive_model_to_data_file,keys(), output_dict=True
)